# Project 2, Implmenting Classification

# 1, Implmenting the decision tree using numpy and pandas without using sk-learn.
# 2, Implementation need include both classification with gini/entropy and regression with rss.
# 3, You are required to test with a sample dataset and provide evaluation with confusion matrix.
# 4, Compare your implementation with Sk-learn regressor.


# 使用基尼不纯度来实现分类树

## 数据集
   
    数据集简介：
        鸢尾花数据集，来自机器学习包sklearn集成的数据集。
    数据集的属性：
        萼片长度(厘米)(sepal length (cm))/萼片长度(厘米)(sepal width (cm))/花瓣长度(厘米)(petal length (cm))
        花瓣长度(厘米)(petal width (cm))/目标(target)/目标名字(target_names)
        上面前4个属性是鸢尾花的特征属性,根据上面的特征属性，可以将划分成相应的目标(0/1/2)，其中目标对应相应的目标名字(0 - setosa),
        (1 - versicolor),(2 - virginica)
    数据集大小：150 * 6
    
## 说明

    1. 加载数据集,并将数据集划分为训练集与测试集
    2. 定义相应的函数
        partition：将数据分割到左边和右边
        gini_impurity：定义基尼不纯度(gini impurity)
        information_gain：定义信息增益(information main)
        find_best_split：定义寻找最佳分割的函数
    3. 定义决策树
        class Leaf：决策树的叶子节点
        Decision_Node：决策树的决策结点(非叶子结点)
    4. 用训练集来训练决策树
        my_tree = build_tree(X_train, y_train, X_train.index)
    5. 用测试集来测试决策树
    6. 用混淆矩阵来评估决策树
    
## 结果
    
    预测的准确度：0.9736842105263158(测试数据有38个，预测错了1个)
    混淆矩阵：
        [[12  0  0]
         [ 0  7  1]
         [ 0  0 18]]
    

In [1]:
import numpy as np 
import pandas as pd 
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from sklearn.model_selection import train_test_split

pd.options.mode.chained_assignment = None

In [2]:
# 加载数据集
dataset = load_iris(as_frame=True)
df= pd.DataFrame(data= dataset.data)
target_zip= dict(zip(set(dataset.target), dataset.target_names))
df["target"] = dataset.target
df["target_names"] = df["target"].map(target_zip)

print(df.shape)
df.head()

(150, 6)


,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target,target_names
0,5.1,3.5,1.4,0.2,0,setosa
1,4.9,3.0,1.4,0.2,0,setosa
2,4.7,3.2,1.3,0.2,0,setosa
3,4.6,3.1,1.5,0.2,0,setosa
4,5.0,3.6,1.4,0.2,0,setosa


In [3]:
# 划分数据集和训练集
X = df.iloc[:, :4]
y = df.iloc[:, -1]

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75, shuffle=True, random_state=24)

## Without using sk-learn.

In [4]:
# 将数据分割到左边和右边
def partition(data, column, value): 
    left = data[data[column] <= value].index
    right = data[data[column] > value].index

    return left, right

In [5]:
# 定义基尼不纯度(gini impurity)
def gini_impurity(label, label_idx):
    unique_label, unique_label_count = np.unique(label.loc[label_idx], return_counts=True)

    impurity = 1.0
    for i in range(len(unique_label)):
        p_i = unique_label_count[i] / sum(unique_label_count)
        impurity -= p_i ** 2 
    return impurity

In [6]:
# 定义信息增益
def information_gain(label, left_idx, right_idx, impurity): 
    p = float(len(left_idx)) / (len(left_idx) + len(right_idx))
    info_gain = impurity - p * gini_impurity(label, left_idx) - (1 - p) * gini_impurity(label, right_idx)
    return info_gain

In [7]:
# 定义寻找最佳分割的函数
def find_best_split(df, label, idx):
    best_gain = 0 
    best_col = None
    best_value = None
    
    df = df.loc[idx] 
    label_idx = label.loc[idx].index 

    impurity = gini_impurity(label, label_idx) 
    
    for col in df.columns: 
        unique_values = set(df[col])
        for value in unique_values: 

            left_idx, right_idx = partition(df, col, value)
            if len(left_idx) == 0 or len(right_idx) == 0: 
                continue 
            info_gain = information_gain(label, left_idx, right_idx, impurity)
            if info_gain > best_gain:
                best_gain, best_col, best_value = info_gain, col, value
                
    return best_gain, best_col, best_value

In [8]:
# 计算唯一值的函数
def count(label, idx):
    unique_label, unique_label_counts = np.unique(label.loc[idx], return_counts=True)
    dict_label_count = dict(zip(unique_label, unique_label_counts))
    return dict_label_count

In [9]:
# 决策树的叶节点
class Leaf:
    def __init__(self, label, idx):
        self.predictions = count(label, idx)

In [10]:
# 决策树的决策结点（非叶子结点）
class Decision_Node:
    def __init__(self, column, value, true_branch, false_branch):
        self.column = column
        self.value = value
        self.true_branch = true_branch
        self.false_branch = false_branch

In [11]:
def build_tree(df, label, idx): 
    best_gain, best_col, best_value = find_best_split(df, label, idx)
    if best_gain == 0: 
        return Leaf(label, label.loc[idx].index)
    left_idx, right_idx = partition(df.loc[idx], best_col, best_value)
    true_branch = build_tree(df, label, left_idx)
    false_branch = build_tree(df, label, right_idx)
    return Decision_Node(best_col, best_value, true_branch, false_branch)

In [12]:
# 打印决策树的函数
def print_tree(node, spacing=""):
    if isinstance(node, Leaf):
        print (spacing + "Predict", node.predictions)
        return
    print(spacing + f"[{node.column} <= {node.value}]")
    print (spacing + '--> True:')
    print_tree(node.true_branch, spacing + "  ")
    print (spacing + '--> False:')
    print_tree(node.false_branch, spacing + "  ")

In [13]:
my_tree = build_tree(X_train, y_train, X_train.index)

In [14]:
print_tree(my_tree)

[petal length (cm) <= 1.9]
--> True:
  Predict {'setosa': 38}
--> False:
  [petal width (cm) <= 1.6]
  --> True:
    [petal length (cm) <= 4.9]
    --> True:
      Predict {'versicolor': 40}
    --> False:
      [sepal length (cm) <= 6.0]
      --> True:
        [sepal width (cm) <= 2.2]
        --> True:
          Predict {'virginica': 1}
        --> False:
          Predict {'versicolor': 1}
      --> False:
        Predict {'virginica': 2}
  --> False:
    [petal length (cm) <= 4.8]
    --> True:
      [sepal width (cm) <= 3.0]
      --> True:
        Predict {'virginica': 3}
      --> False:
        Predict {'versicolor': 1}
    --> False:
      Predict {'virginica': 26}


In [15]:
def predict(test_data, tree):
    if isinstance(tree, Leaf): 
        return max(tree.predictions)
    feature_name, feature_value = tree.column, tree.value
    if test_data[feature_name] <= feature_value: 
        return predict(test_data, tree.true_branch)
    else: 
        return predict(test_data, tree.false_branch)

In [16]:
# 创造新的预测列
X_test["predictions"] = X_test.apply(predict, axis=1, args=(my_tree,))

In [17]:
# 计算并打印预测的准确度
print(f"My Implementation:\nACCURACY: {accuracy_score(y_test, X_test['predictions'])}")

My Implementation:
ACCURACY: 0.9736842105263158


In [18]:
# 用混淆矩阵来评估模型

In [19]:
from sklearn.metrics import confusion_matrix

In [20]:
cm = confusion_matrix(y_test, X_test['predictions'])
print('Confusion matrix\n\n', cm)

Confusion matrix

 [[12  0  0]
 [ 0  7  1]
 [ 0  0 18]]


In [21]:
# 通过混淆矩阵来获取准确度
accuracy = (cm[0][0] + cm[1][1] + cm[2][2]) / y_test.shape[0]
print(accuracy)

0.9736842105263158
